# Varying diameter

In this notebook we will try to create a scaling law to account for a varying propeller diameter as well as a varying number of blade and a varying altitude. Twist law should also vary but for now it will be left as is.

In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

We'll adjust slightly the defined set here to take something similar to the previous notebook, that is $\pi_2 = \frac{picth}{PITCH_{REF}}$ and $\pi_3 = n_{blade}$

In [ ]:
import pandas as pd
import numpy as np

results_fixed_pitch = pd.read_csv("data/doe_fixed_pitch.csv")
results_ICE = pd.read_csv("data/doe_ICE.csv")
results_TP = pd.read_csv("data/doe_TP.csv")

results = pd.concat([results_fixed_pitch, results_ICE, results_TP])

number_of_element = len(results["Thrust"].to_numpy())

print("Number of element pre screening: ", number_of_element)

sampling = 1

thrust_array = results["Thrust"].to_numpy()[0::sampling]
power_array = results["Power"].to_numpy()[0::sampling]
eta_array = results["Efficiency"].to_numpy()[0::sampling]
sigma_array = results["Solidity"].to_numpy()[0::sampling]
AF_array = results["Activity Factor"].to_numpy()[0::sampling]
lambda_array = results["Aspect ratio"].to_numpy()[0::sampling]
n_rot_val = results["Rotational speed"].to_numpy()[0::sampling]
D_val = results["Diameter"].to_numpy()[0::sampling]
n_blade_val = results["Number of blades"].to_numpy()[0::sampling]
RHO_val = results["Density"].to_numpy()[0::sampling]
pitch_val = results["Pitch at 0.75R"].to_numpy()[0::sampling]
PITCH_REF_val = results["Ref Pitch at 0.75R"].to_numpy()[0::sampling]
v_val = results["Flight speed"].to_numpy()[0::sampling]
altitude = results["Altitude"].to_numpy()[0::sampling]
twist_blade = results["Blade twist between root and tip"].to_numpy()[0::sampling]

print(np.min(n_rot_val), np.max(n_rot_val))
print(np.min(D_val), np.max(D_val))
print(np.min(RHO_val), np.max(RHO_val))
print(np.min(v_val), np.max(v_val))
print(np.min(n_blade_val), np.max(n_blade_val))

print("Number of element post screening: ", len(n_rot_val))

ct_array = thrust_array / (RHO_val * n_rot_val**2 * D_val**4)
cp_array = power_array / (RHO_val * n_rot_val**3 * D_val**5)

In [ ]:
from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = cp_array
pi7 = AF_array
pi8 = lambda_array
pi9 = twist_blade
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8, pi9]
doe_ct = pd.DataFrame(
    doe_ct, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

In [ ]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter_matrix(doe_ct, width=1200, height=800)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
scatter = go.Scatter(x=AF_array, y=lambda_array, mode="markers", name="AF as a function of lamda")
# fig.add_trace(scatter)
# scatter = go.Scatter(
#         x=lambda_array, y=10000*3/39/lambda_array, mode="markers", name="AF as a function of lamda"
# )
fig.add_trace(scatter)
fig.update_layout(width=1200, height=800)
fig.show()

# Cruise conditions ($\eta$ from 0.75 to 0.95 and J > 1.0)

In [ ]:
import plotly.express as px
from functools import reduce

cruise_idx_1 = np.where(eta_array >= 0.75)
cruise_idx_2 = np.where(pi2 > 1.0)

cruise_idx = reduce(
    np.intersect1d,
    (
        cruise_idx_1,
        cruise_idx_2,
    ),
)

print(len(cruise_idx_1[0]))
print(len(cruise_idx))

pi1_cruise = pi1[cruise_idx]
pi2_cruise = pi2[cruise_idx]
pi3_cruise = pi3[cruise_idx]
pi4_cruise = pi4[cruise_idx]
pi5_cruise = pi5[cruise_idx]
pi6_cruise = pi6[cruise_idx]
pi7_cruise = pi7[cruise_idx]
pi8_cruise = pi8[cruise_idx]
pi9_cruise = pi9[cruise_idx]
doe_ct_cruise = np.c_[
    pi1_cruise,
    pi2_cruise,
    pi3_cruise,
    pi4_cruise,
    pi5_cruise,
    pi6_cruise,
    pi7_cruise,
    pi8_cruise,
    pi9_cruise,
]
doe_ct_cruise = pd.DataFrame(
    doe_ct_cruise, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

df = px.data.iris()
fig = px.scatter_matrix(doe_ct_cruise, width=1200, height=800)
fig.show()

# Low speed conditions ($\eta$ from 0.5 to 0.7 and J < 1.0)

In [ ]:
import plotly.express as px
from functools import reduce

ls_idx_1 = np.where(eta_array >= 0.5)
ls_idx_2 = np.where(eta_array < 0.75)
ls_idx_3 = np.where(pi2 < 1.0)

ls_idx = reduce(
    np.intersect1d,
    (
        ls_idx_1,
        ls_idx_2,
        ls_idx_3,
    ),
)

pi1_ls = pi1[ls_idx]
pi2_ls = pi2[ls_idx]
pi3_ls = pi3[ls_idx]
pi4_ls = pi4[ls_idx]
pi5_ls = pi5[ls_idx]
pi6_ls = pi6[ls_idx]
pi7_ls = pi7[ls_idx]
pi8_ls = pi8[ls_idx]
pi9_ls = pi9[ls_idx]
doe_ct_ls = np.c_[pi1_ls, pi2_ls, pi3_ls, pi4_ls, pi5_ls, pi6_ls, pi7_ls, pi8_ls, pi9_ls]
doe_ct_ls = pd.DataFrame(
    doe_ct_ls, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

df = px.data.iris()
fig = px.scatter_matrix(doe_ct_ls, width=1200, height=800)
fig.show()

# Low efficiency conditions ($\eta$ less than 0.5)

In [ ]:
import plotly.express as px
from functools import reduce

le_idx = np.where(eta_array < 0.5)

pi1_le = pi1[le_idx]
pi2_le = pi2[le_idx]
pi3_le = pi3[le_idx]
pi4_le = pi4[le_idx]
pi5_le = pi5[le_idx]
pi6_le = pi6[le_idx]
pi7_le = pi7[le_idx]
pi8_le = pi8[le_idx]
pi9_le = pi9[le_idx]
doe_ct_le = np.c_[pi1_le, pi2_le, pi3_le, pi4_le, pi5_le, pi6_le, pi7_le, pi8_le, pi9_le]
doe_ct_le = pd.DataFrame(
    doe_ct_le, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

df = px.data.iris()
fig = px.scatter_matrix(doe_ct_le, width=1200, height=800)
fig.show()

# Including the efficiency

In [ ]:
from stdatm import Atmosphere
import plotly.express as px

atm = Atmosphere(altitude, altitude_in_feet=False)

pi0 = eta_array
pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = cp_array
pi7 = AF_array
pi8 = lambda_array
pi9 = twist_blade
doe_ct = np.c_[pi0, pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8, pi9]
doe_ct = pd.DataFrame(
    doe_ct, columns=["eta", "Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

df = px.data.iris()
fig = px.scatter_matrix(doe_ct, width=1200, height=800)
fig.show()

# New DOE

In [ ]:
from stdatm import Atmosphere
import plotly.express as px

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = (
    np.sqrt(v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0)
    * D_val
    / atm.kinematic_viscosity
)
pi5 = sigma_array
pi6 = cp_array
pi7 = AF_array
pi8 = lambda_array
pi9 = twist_blade
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8, pi9]
doe_ct = pd.DataFrame(
    doe_ct, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "lambda", "Theta_b"]
)

df = px.data.iris()
fig = px.scatter_matrix(doe_ct, width=1200, height=800)
fig.show()